In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [ ]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete9HS7SY/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete9HS7SY/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete9HS7SY/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
for s, l in imdb['train']: 
    print(s)
    print(l)
    break

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)
tf.Tensor(0, shape=(), dtype=int64)


In [ ]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s, l in train_data:
    training_sentences.append(s.numpy().decode('utf8'))
    training_labels.append(l.numpy())

for s, l in test_data:
    testing_sentences.append(s.numpy().decode('utf8'))
    testing_labels.append(l.numpy())

In [ ]:
training_labels = np.array(training_labels)
testing_labels = np.array(testing_labels)

In [ ]:
training_sentences[:10]

["This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
 'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development wa

In [ ]:
training_labels[:10]

[0, 0, 0, 1, 1, 1, 0, 0, 0, 0]

In [ ]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = '<oov>'

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen = max_length, truncating = trunc_type)

testing_sequence = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequence, maxlen = max_length, truncating = trunc_type)

In [ ]:
reverse_word_index  = dict([(value, key) for (key, value) in word_index.items()])
for i, values in enumerate(reverse_word_index.items()):
    print(values)
    if i > 10:
        break

(1, '<oov>')
(2, 'the')
(3, 'and')
(4, 'a')
(5, 'of')
(6, 'to')
(7, 'is')
(8, 'br')
(9, 'in')
(10, 'it')
(11, 'i')
(12, 'this')


In [ ]:
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text]) # error 나오는 인덱스를 '?' 로 표시

print(decode_review(padded[3]))
print(padded)
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <oov> into a big arm chair and <oov> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <oov> to cross no dangerous waters just a warm and witty <oov> through new york life at its best a family film in every sense and one that deserves the praise it received
[[   0    0    0 ...  867  141   10]
 [   0    0    0 ...   20   31   30]
 [4383 6109    2 ...  550    5  735]
 ...
 [ 861   36   11 ...    8    8    2]
 [  52   11  217 ...  251   94   42]
 [  84    5   30 ...  131    7    6]]
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row

In [ ]:
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
                             tf.keras.layers.Conv1D(128, 5, activation = 'relu'),
                             tf.keras.layers.GlobalAveragePooling1D(),
                             tf.keras.layers.Dense(64, activation = 'relu'),
                             tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [ ]:
model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
NUM_EPOCHS = 10

history = model.fit(padded, training_labels, validation_data=(testing_padded, testing_labels),
                    epochs=NUM_EPOCHS)

Epoch 1/10
782/782 [==============================] - 6s 8ms/step - loss: 0.0263 - accuracy: 0.9917 - val_loss: 1.2996 - val_accuracy: 0.7910
Epoch 2/10
782/782 [==============================] - 6s 7ms/step - loss: 0.0236 - accuracy: 0.9924 - val_loss: 1.3600 - val_accuracy: 0.7860
Epoch 3/10
782/782 [==============================] - 6s 7ms/step - loss: 0.0166 - accuracy: 0.9952 - val_loss: 1.5755 - val_accuracy: 0.7890
Epoch 4/10
782/782 [==============================] - 6s 7ms/step - loss: 0.0154 - accuracy: 0.9946 - val_loss: 1.7113 - val_accuracy: 0.7858
Epoch 5/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0117 - accuracy: 0.9952 - val_loss: 1.9489 - val_accuracy: 0.7841
Epoch 6/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0119 - accuracy: 0.9956 - val_loss: 1.9017 - val_accuracy: 0.7910
Epoch 7/10
782/782 [==============================] - 6s 7ms/step - loss: 0.0068 - accuracy: 0.9978 - val_loss: 2.0643 - val_accuracy: 0.7815
Epoch 